# Handgeschriebene Ziffern

Der beiliegenden Beschreibung der Daten habe ich entnommen, dass es sich um handgeschriebene Ziffern handelt. Die mit 16x16 Bildpunkten abgespeichert sind. Der Beschreibung kann ich weiter entnehmen:

*"The data are in two gzipped files, and each line consists of the digit
id (0-9) followed by the 256 grayscale values."*

Es handelt sich also um eine Textdatei in der in jeder Zeile der Wert der geschriebenen Ziffer sowie die 16x16 = 256 Bildpunkte stehen.

---

In [ ]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

---

In [ ]:
def getNumber(data, digit):
    # wähle alle Zeilen in denen die erste Spalte (Spalte mit den Ziffer) mit 'digit' übereinstimmt
    data_number = pd.DataFrame(data[data[0] == digit]) 
    # schneide die erste Spalte weg und gebe den rest zurück
    separated_data = data_number.iloc[:,1:]
    return separated_data


In [ ]:
def getNumberImage(data, digit, aggregation):
    # ruft die Hilfsfunktion getNumber auf um alle Bilddaten Daten zu einer Ziffer zu holen
    image_data = getNumber(data, digit)
    # bilde aus allen Bildern zu der einen Ziffer ein aggrigiertes Bild
    if(aggregation == 'median'):
        df1 = pd.DataFrame(image_data.median())
    else:
        df1 = pd.DataFrame(image_data.mean())        
    
    # wandle das format von (1,256) -> (16,16) und gebe diesen transformierten DataFrame zurück
    return df1.values.reshape(16,16)

In [ ]:
class Experiment:
    def __init__(self, targets_train, targets_predict):
        self.results_train = pd.DataFrame(targets_train, columns=["targets"])
        self.results_predict = pd.DataFrame(targets_predict, columns=["targets"])

        self.input_data = {}
        
        self.names = []        
        self.configs = []

        
    def get_names(self):
        return self.names
    
    
    def get_results(self):
        return self.results_train, self.results_predict
    
    def get_config_map(self):
        for conf in self.configs:
            print ("CONFIG:", conf['name'], conf['input_data_name'], "\tNum features:", self.input_data[conf['input_data_name']]['data'].shape[1])
    
    
    def add_input_data(self, name, data, description=""):
        self.input_data[name] = {'description': description, 'data': data}
    
    
    def add_config(self, algo, name, input_data_name):
        # TODO: add check for dublicate names
        if input_data_name not in self.input_data:
            print ("ERROR: feature_set_name not found")
            return 
        
        self.names.append(name)
        self.configs.append({ 'name': name, 'algo': algo, 'input_data_name': input_data_name, 'fit': -1})
        

    def _fit(self, conf):
        # get features
        input_data = self.input_data[conf['input_data_name']]['data']

        # fit the features
        print ("Fitting ", conf['name'], "... ")
        
        start = time.time() # startzeit
        conf['fit'] = conf['algo'].fit(input_data, self.results_train['targets'])
        end = time.time() # startzeit
        
        print ("done in ", end - start , "s.")

        
    
    def _predict(self, conf, data, train):
        if data.shape[1] != self.input_data[conf['input_data_name']]['data'].shape[1]:
            print ("ERROR: Number of input features does not match (", conf['name'], ")")
            return
        
        print ("Predicting data with", conf['name'], "... ")
        
        start = time.time() # startzeit
        if train:
            self.results_train[conf['name']] = conf['algo'].predict(data)
        else:
            self.results_predict[conf['name']] = conf['algo'].predict(data)
        end = time.time() # startzeit
        
        print ("done in ", end - start , "s." )

        
    def fit(self):
        '''Only trains algorithms if no fit has been calculated before'''
        for conf in self.configs:
            if conf['fit'] == -1:
                self._fit(conf)

                
    def fit_all(self, data):
        '''Trains all algorithms with given data'''
        for conf in self.configs:
            self._fit(conf)

            
    def refit(self, data):
        '''Retrains all algorithms with given data that have been trained before'''
        for conf in self.configs:
            if conf['fit'] != -1:
                self._fit(conf)

                
    def predict(self, name, data, train=False):
        for conf in self.configs:
            if conf['fit'] != -1 and conf['name'] == name:
                self._predict(conf, data, train)

    def predict_all(self, data, train=False):
        for conf in self.configs:
            if conf['fit'] != -1:
                self._predict(conf, data, train)

                
    def get_count_df(self, name):
        count_matrix = np.zeros((10,10)) 
        for digit in range(0,10):
            cluster_counts = self.results_train[self.results_train['targets'] == digit].groupby(name).count()['targets']
            for cluster in cluster_counts.keys():
                count_matrix[digit][cluster] = cluster_counts[cluster]
        
        col_names = []
        for i in range(0,10):
            col_names.append("C_" + str(i))
    
        return pd.DataFrame(count_matrix, columns=col_names)

    def get_norm_df(self, name):
        count_df = self.get_count_df(name)
        return count_df.divide(count_df.sum(1), axis=0)
    
    
    def get_all_error_rates(self):
        error_rates = {}
        for conf in self.configs:
            if (conf['fit'] != -1) & (conf['name'] in self.results_train.columns):
                error_rates[conf['name']] = self.get_error_rate(conf['name'])
        return error_rates
                
    def get_error_rate(self, name):
        '''Determine classification error by identifying best fitting column assignment'''
       
            

---

In [ ]:
# mit header=None, wird die erste Zeile der Datei nicht als Header interpretiert 
#              (Man könnte den Header in einem solchen Fall als Zeile mit den Spaltenüberschriften bezeichnen)
# mit sep=" ", geben wir an, dass wir das Leerzeichen als Seperator verwenden wollen. 
#              D.h. zwei durch ein Leerzeichen separierte Werte sollen als zwei Werte eingelesen werden.
data_train = pd.read_csv("../data/zip.train", header=None, sep=" ") 

In [ ]:
cleaned_train_data = data_train.dropna(axis=1, thresh=2) # lass alle Spalten mit mehr als 2 NaN (Not a Number) vom datensatz fallen 
cleaned_train_data.shape

In [ ]:
cleaned_train_data.describe()

Aus der Beschreibung der Daten wissen wir, dass es sich um die Ziffern und die dazugehörigen Bilder der Größe 16x16 Pixel handelt. Also in jeder Zeile steht in der 0-ten Spalte die Ziffer und in den folgenden 256 Spalten die einzelnen Pixel der Bilder. Also visualisieren wir diese nun einmal.

In [ ]:
plt.figure(1, figsize=(20, 10))
for i in range(0,10):
    image = getNumberImage(cleaned_train_data,i,'mean')

    # Call signature: subplot(nrows, ncols, index, **kwargs)
    plt.subplot(2,5, 1 + i)
    plt.imshow(image, cmap='hot', interpolation='none')

plt.show()

---

In [ ]:
data_test = pd.read_csv("../data/zip.test", header=None, sep=" ") 

In [ ]:
cleaned_test_data = data_test.dropna(axis=1, thresh=2) # lass alle Spalten mit mehr als 2 NaN (Not a Number) vom datensatz fallen 
cleaned_test_data.shape

In [ ]:
cleaned_test_data.describe()

In [ ]:
plt.figure(1, figsize=(20, 10))
for i in range(0,10):
    image = getNumberImage(cleaned_test_data,i,'mean')

    # Call signature: subplot(nrows, ncols, index, **kwargs)
    plt.subplot(2,5, 1 + i)
    plt.imshow(image, cmap='hot', interpolation='none')

plt.show()

---

## Split data in train und test

In [ ]:
# train data
input_data = cleaned_train_data.iloc[:,1:].values
input_data.shape

In [ ]:
# test data
test_data = cleaned_test_data.iloc[:,1:].values
test_data.shape

In [ ]:
# Set targets
exp = Experiment(cleaned_train_data[0].values, cleaned_test_data[0].values)

### Build features

In [ ]:
# input featurs are the individual pixels of the image - without transformation
exp.add_input_data('256_pixel', input_data)

## Classification mit KNN

Wir nehmen an, dass jeder Bildpunkt ein Wert in dem 256 dimensionalen Raum der reelen Zahlen ist. In diesem Raum haben wir wie im 1-, 2-, oder 3-Dimensionalen auch das euklidische Abstandsmaß.

In [ ]:
exp.add_config(KNeighborsClassifier(n_neighbors=1), 'KNN_1', '256_pixel')

-----

## Classification with logistic regression

In [ ]:
exp.add_config(LogisticRegression(n_jobs=1), 'LogReg_1', '256_pixel')


## Training

In [ ]:
print(exp.get_names())
print(exp.get_config_map())

In [ ]:
exp.fit()

## Vorhersage und Fehler der Verfahren auf dem Test set

In [ ]:
exp.predict_all(input_data, train=True)
exp.predict_all(test_data)

In [ ]:
results_train, results_test = exp.get_results()
results_train.head()

In [ ]:
results_test.head()

In [1]:
print("KNR:",1.0*(results_test['targets'] == results_test['KNN_1']).sum()/results_test['targets'].count())
print("LogReg:",1.0*(results_test['targets'] == results_test['LogReg_1']).sum()/results_test['targets'].count())

NameError: name 'results_test' is not defined